# Chains in LangChain
## Outline
* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [ ]:
import os
os.environ["TOGETHER_API_KEY"]="YOUR+TOGETHER_AI_KEY"

In [61]:
#!pip install --upgrade langchain
#%pip install -qU langchain-together
#!pip install together
#!pip install -U langchain-community
#!pip install -U langchain
#!pip install pandas

In [62]:
import pandas as pd
df=pd.read_csv('Data.csv')

In [63]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [64]:
df.tail()

,Product,Review
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


# LLMChain

In [65]:
from together import Together
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_together import ChatTogether
llm=ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    temperature=0.9
)

In [66]:
prompt=ChatPromptTemplate.from_template(
    "Suggest the best company name for a business that makes {product}. Respond with only the name."
)

In [67]:
chain=LLMChain(llm=llm,prompt=prompt)

In [68]:
product="Queen Size Sheet Set"
chain.run(product)

'Regal Slumber Co.'

In [69]:
chain.invoke(product)

{'product': 'Queen Size Sheet Set', 'text': 'RegalSlumber Co.'}

# SimpleSequentialChain
Single Input-Single Output

In [77]:
from langchain.chains import SimpleSequentialChain
# prompt template 1
first_prompt = ChatPromptTemplate.from_template("What is the best name to describe \ a company that makes {product}?Respond with only the name.")
# Chain 1
chain_one=LLMChain(llm=llm, prompt=first_prompt)
# prompt template 2
second_prompt=ChatPromptTemplate.from_template("Write a 20 words description for the following \company:{company_name}")
# chain 2
chain_two=LLMChain(llm=llm,prompt=second_prompt)

In [78]:
overall_simple_chain=SimpleSequentialChain(chains=[chain_one, chain_two],verbose=True)

In [79]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Regal Slumber Co.
Regal Slumber Co. offers luxurious mattresses and bedding for a restful, rejuvenating sleep experience every night guaranteed always.

> Finished chain.


'Regal Slumber Co. offers luxurious mattresses and bedding for a restful, rejuvenating sleep experience every night guaranteed always.'

# SequentialChain
Multiple Input Multiple Output

In [80]:
from langchain.chains import SequentialChain
# prompt template 1: translate to english
first_prompt=ChatPromptTemplate.from_template("Translate the following review to english:""\n\n{Review}"
)
# chain 1:input=Review and output=English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt,output_key="English_Review")

In [81]:
second_prompt=ChatPromptTemplate.from_template("Can you summarize the following review in 1 sentence:""\n\n{English_Review}")
# chain 2: input=English_Review and output=summary
chain_two=LLMChain(llm=llm, prompt=second_prompt,output_key="summary")

In [82]:
# prompt template 3:translate to english
third_prompt=ChatPromptTemplate.from_template("What language is the following review:\n\n{Review}")
# chain 3:input=Review and output=language
chain_three=LLMChain(llm=llm, prompt=third_prompt,output_key="language")

In [84]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template("Write a follow up response to the following ""summary in the specified language:""\n\nSummary: {summary}\n\nLanguage: {language}")
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,output_key="followup_message")

In [85]:
# overall_chain: input= Review
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(chains=[chain_one, chain_two, chain_three, chain_four],input_variables=["Review"],output_variables=["English_Review", "summary","followup_message"],verbose=True)

In [86]:
df.Review[5]

"Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?"

In [87]:
r=df.Review[5]
overall_chain(r)

<ipython-input-87-d85fd42dda85>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(r)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': 'Here\'s the translation of the review to English:\n\n"I find the taste mediocre. The foam doesn\'t hold, it\'s weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!? "\n\nNote: The reviewer seems to be questioning the authenticity of the product, suggesting that it might be an old batch or a counterfeit item, which could explain the poor taste and texture.',
 'summary': "The reviewer was disappointed with the product, finding the taste mediocre and the foam unstable, and suspects that it may be an old batch or counterfeit item, as it differs significantly from the same product they've purchased in stores.",
 'followup_message': "Je partage entièrement l'avis de ce critique, ayant moi-même été déçu par le produit en question. Le goût, loin d'être excep

In [88]:
r=df.Review[1]
overall_chain(r)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
 'English_Review': "The translation is already provided in English. However, to rephrase it for clarity:\n\nI really liked the waterproof bag, but I had some concerns about the opening being made of hard plastic, as it might be prone to breaking. Additionally, I encountered an issue where I couldn't turn on my phone once it was inside the pouch.",
 'summary': 'The reviewer liked the waterproof bag but had concerns about its durability due to the hard plastic opening and experienced an issue with their phone not turning on while inside the pouch.',
 'followup_message': "I appreciate your feedback on our waterproof bag, and I'm glad to hear that you liked its overall design. However, I apologize for the concerns you've raised regarding its durability, particularly with the hard plastic opening. I und

# Router Chain

In [90]:
physics_template="""You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""

math_template="""You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template="""You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

computerscience_template=""" You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity.

Here is a question:
{input}"""

In [91]:
prompt_infos=[
    {
        "name":"physics",
        "description":"Good for answering questions about physics",
        "prompt_template":physics_template
    },
    {
        "name":"math",
        "description":"Good for answering math questions",
        "prompt_template":math_template
    },
    {
        "name":"History",
        "description":"Good for answering history questions",
        "prompt_template":history_template
    },
    {
        "name":"computer science",
        "description":"Good for answering computer science questions",
        "prompt_template":computerscience_template
    }
]

In [92]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [95]:
destination_chains={}
for p_info in prompt_infos:
    name=p_info["name"]
    prompt_template=p_info["prompt_template"]
    prompt=ChatPromptTemplate.from_template(template=prompt_template)
    chain=LLMChain(llm=llm, prompt=prompt)
    destination_chains[name]=chain

destinations=[f"{p['name']}:{p['description']}" for p in prompt_infos]
destinations_str="\n".join(destinations)

In [96]:
default_prompt=ChatPromptTemplate.from_template("{input}")
default_chain=LLMChain(llm=llm, prompt=default_prompt)

In [97]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to wrap the output with ```json (output)```)>>"""

In [98]:
router_template=MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt=PromptTemplate(template=router_template,input_variables=["input"],output_parser=RouterOutputParser())
router_chain=LLMRouterChain.from_llm(llm,router_prompt)

In [99]:
chain=MultiPromptChain(router_chain=router_chain,destination_chains=destination_chains,default_chain=default_chain,verbose=True)

<ipython-input-99-7a33160ca4f8>:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain=MultiPromptChain(router_chain=router_chain,destination_chains=destination_chains,default_chain=default_chain,verbose=True)


In [100]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation is a fundamental concept in physics. It refers to the thermal electromagnetic radiation emitted by an object at a certain temperature. The term "black body" doesn\'t mean the object is actually black, but rather that it\'s a perfect absorber of electromagnetic radiation, meaning it absorbs all the radiation that hits it and doesn\'t reflect any.\n\nWhen an object is heated, its atoms or molecules vibrate and collide with each other, causing them to release excess energy in the form of electromagnetic radiation. The radiation emitted by a black body is characterized by a specific spectrum, which depends only on its temperature, not on its composition or other properties.\n\nThe key feature of black body radiation is that it follows a predictable pattern, described by Planck\'s law, which relates the intensity of the radiation to the temperature of the object and the wavelength of the radiation. This law was formulated by Max Planck in 1900 and marked the beginning 

In [101]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'To answer this question, let\'s break it down into its simplest component parts.\n\nThe question is asking for the result of adding 2 and 2 together. \n\nThe component parts of this question are:\n1. The number 2\n2. The operation of addition (denoted by the plus sign, +)\n3. The number 2 (again, since we\'re adding 2 to 2)\n\nNow, let\'s analyze each component part:\n1. We know that the number 2 is a positive integer.\n2. The operation of addition means we are looking to combine the values of the two numbers.\n3. The second number 2 is also a positive integer, just like the first one.\n\nNow, let\'s put these component parts together. When we add 2 and 2, we are essentially counting how many units we have in total. \n\n2 (initial units) + 2 (additional units) = 4 units\n\nTherefore, the answer to the question "what is 2 + 2" is 4.'

In [102]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in the human body contain DNA from a biological perspective?'}
> Finished chain.


"From a biological perspective, every cell in the human body contains DNA because it is the primary genetic material that carries the instructions for the development, growth, and function of an organism. DNA (Deoxyribonucleic acid) is a long, double-stranded molecule that contains the genetic code, which is made up of four nucleotide bases - adenine (A), guanine (G), cytosine (C), and thymine (T).\n\nHere are some reasons why every cell in the human body contains DNA:\n\n1. **Genetic inheritance**: DNA is passed down from one generation to the next, and it contains the genetic information that defines an individual's traits, characteristics, and susceptibility to certain diseases. Every cell in the body receives a copy of the DNA molecule during cell division, ensuring that the genetic information is preserved and transmitted to daughter cells.\n2. **Cellular differentiation**: DNA contains the instructions for cellular differentiation, which is the process by which a cell becomes spe